In [3]:
using JuMP, Gurobi, XLSX

In [4]:
xf = XLSX.readxlsx("toy-stores.xlsx")

XLSXFile("toy-stores.xlsx") containing 1 Worksheet
            sheetname size          range        
-------------------------------------------------
               Sheet1 102x12        A1:L102      


In [5]:
data = xf["Sheet1"]

102×12 XLSX.Worksheet: ["Sheet1"](A1:L102)

In [6]:
model = Model(with_optimizer(Gurobi.Optimizer))
M = 100
N = 24

Academic license - for non-commercial use only


24

In [7]:
set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "Heuristics", 0)
set_optimizer_attribute(model, "Cuts", 0)

0

In [22]:
# Variables
@variable(model, 0 <= x[1:M,1:N]) # + Constraint (4)
@variable(model, 0 <= y[1:N],Bin) # + Constraint (5)

LoadError: An object of name x is already attached to this model. If this is intended, consider using the anonymous construction syntax, e.g., x = @variable(model, [1:N], ...) where the name of the object does not appear inside the macro.

In [23]:
# Compute costs C_ij and fixed costs f_j
C = zeros(Float64, (M,N))
f = zeros(Float64,N)
for j in 1:N
    lat2 = deg2rad(data[j+2,11])
    long2 = deg2rad(data[j+2,10])
    f[j]=data[j+2,12]
    for i in 1:M
        lat = deg2rad(data[i+2,4]) 
        long = deg2rad(data[i+2,3])
        if lat==lat2 && long ==long2 
            C[i,j]=0
        else
            C[i,j]=2*asin(sqrt( sin((lat2-lat)/2)^2 + cos(lat)*cos(lat2)*sin((long2-long)/2)^2 ))*3958.8 *data[i+2,5]
            #C[i,j]= acos(sin(lat)*sin(lat2)+cos(lat)*cos(lat2)*cos(long-long2))
        end
    end
end

In [24]:
# Objective function
@objective(model, Min, sum(C .* x) + sum(f .* y))

9.945386429637533e6 x[1,1] + 1.0916440001080781e7 x[3,1] + 1.529713000471215e7 x[4,1] + 2.8584887753416053e6 x[5,1] + 7.636521417379292e6 x[6,1] + 5.719058723359763e6 x[7,1] + 4.2088440734258115e7 x[8,1] + 9.30615403999812e6 x[9,1] + 5.10685747007257e6 x[10,1] + 4.0490804487175584e7 x[11,1] + 7.826941172766576e6 x[12,1] + 1.738843354926437e7 x[13,1] + 1.9901036338442277e7 x[14,1] + 7.936849835771717e6 x[15,1] + 8.540213115409872e7 x[16,1] + 1.570651967564273e7 x[17,1] + 2.4488142996159937e7 x[18,1] + 3.194866646230881e6 x[19,1] + 1.2328849531384233e7 x[20,1] + 7.334404450811219e6 x[21,1] + 7.617069791690432e6 x[22,1] + 6.967495242111729e6 x[23,1] + 4.718677991244333e6 x[24,1] + 6.001098097342096e6 x[25,1] + 1.0140859850633493e7 x[26,1] + 7.1681493103836775e6 x[27,1] + 4.04820894661242e6 x[28,1] + 3.1107184843962104e6 x[29,1] + 7.42899415583295e6 x[30,1] + 9.95130375095974e6 x[31,1] + 6.614547456944196e6 x[32,1] + 7.648822610608423e6 x[33,1] + 7.735959017487958e6 x[34,1] + 8.06544006777

In [25]:
# Constraint (2)
@constraint(model, sum(x, dims=2) .== ones(Float64, M))

100×1 Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}},ScalarShape},2}:
 x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[1,6] + x[1,7] + x[1,8] + x[1,9] + x[1,10] + x[1,11] + x[1,12] + x[1,13] + x[1,14] + x[1,15] + x[1,16] + x[1,17] + x[1,18] + x[1,19] + x[1,20] + x[1,21] + x[1,22] + x[1,23] + x[1,24] = 1.0
 x[2,1] + x[2,2] + x[2,3] + x[2,4] + x[2,5] + x[2,6] + x[2,7] + x[2,8] + x[2,9] + x[2,10] + x[2,11] + x[2,12] + x[2,13] + x[2,14] + x[2,15] + x[2,16] + x[2,17] + x[2,18] + x[2,19] + x[2,20] + x[2,21] + x[2,22] + x[2,23] + x[2,24] = 1.0
 x[3,1] + x[3,2] + x[3,3] + x[3,4] + x[3,5] + x[3,6] + x[3,7] + x[3,8] + x[3,9] + x[3,10] + x[3,11] + x[3,12] + x[3,13] + x[3,14] + x[3,15] + x[3,16] + x[3,17] + x[3,18] + x[3,19] + x[3,20] + x[3,21] + x[3,22] + x[3,23] + x[3,24] = 1.0
 x[4,1] + x[4,2] + x[4,3] + x[4,4] + x[4,5] + x[4,6] + x[4,7] + x[4,8] + x[4,9] + x[4,10] + x[4,11] + x[4,12] + x[4,13] + x[4,14

In [26]:
# Constraint (3)
@constraint(model, [i = 1:M], x[i,:] .<= y)

100-element Array{Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1},1}:
 [x[1,1] - y[1] ≤ 0.0, x[1,2] - y[2] ≤ 0.0, x[1,3] - y[3] ≤ 0.0, x[1,4] - y[4] ≤ 0.0, x[1,5] - y[5] ≤ 0.0, x[1,6] - y[6] ≤ 0.0, x[1,7] - y[7] ≤ 0.0, x[1,8] - y[8] ≤ 0.0, x[1,9] - y[9] ≤ 0.0, x[1,10] - y[10] ≤ 0.0  …  x[1,15] - y[15] ≤ 0.0, x[1,16] - y[16] ≤ 0.0, x[1,17] - y[17] ≤ 0.0, x[1,18] - y[18] ≤ 0.0, x[1,19] - y[19] ≤ 0.0, x[1,20] - y[20] ≤ 0.0, x[1,21] - y[21] ≤ 0.0, x[1,22] - y[22] ≤ 0.0, x[1,23] - y[23] ≤ 0.0, x[1,24] - y[24] ≤ 0.0]
 [x[2,1] - y[1] ≤ 0.0, x[2,2] - y[2] ≤ 0.0, x[2,3] - y[3] ≤ 0.0, x[2,4] - y[4] ≤ 0.0, x[2,5] - y[5] ≤ 0.0, x[2,6] - y[6] ≤ 0.0, x[2,7] - y[7] ≤ 0.0, x[2,8] - y[8] ≤ 0.0, x[2,9] - y[9] ≤ 0.0, x[2,10] - y[10] ≤ 0.0  …  x[2,15] - y[15] ≤ 0.0, x[2,16] - y[16] ≤ 0.0, x[2,17] - y[17] ≤ 0.0, x[2,18] - y[18] ≤ 0.0, x[2,19] - y[19] ≤ 0.0, x[2,20] - y[20] ≤ 0.0, x[2,21] - y[21] ≤ 

In [27]:
# Constraint (6)
#@constraint(model, sum(x, dims=1) .<= M * transpose(y))

In [28]:
optimize!(model)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 7400 rows, 2424 columns and 19200 nonzeros
Model fingerprint: 0xf256bfc7
Variable types: 2400 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+04, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

MIP start from previous solve produced solution with objective 4.47007e+08 (0.03s)
Loaded MIP start from previous solve with objective 4.47007e+08

Variable types: 2400 continuous, 24 integer (24 binary)

Root relaxation: cutoff, 189 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      4.4701e+08 4.4701e+08  0.00%     -    0s

Explored 0 nodes (189 simplex iterations) in 0.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 4.47007e+08 

Optimal 

In [29]:
findall(x->x!=-0.0, JuMP.value.(y))

7-element Array{Int64,1}:
  5
  8
 10
 12
 14
 16
 24

A savoir : Buffalo, Houston, Minneapolis, Oakland, Orlando, Paradise CDP, Winston-Salem

In [30]:
xsum = sum(JuMP.value.(x),dims=1)
xsum[findall(x->x!=0.0,xsum)]

7-element Array{Float64,1}:
 19.0
 15.0
  9.0
  8.0
  9.0
 20.0
 20.0